# Lesson 2 - User Intent

With the fundamentals of agent development with Google ADK in mind, you can start to build a workflow
for knowledge graph construction. 

First things first:
- Knowledge graph construction begins like any data modeling exercise -- the first step is to figure out motivation. 
- What do you want to do with the data?
- This is important for normal data engineering, and critical for designing an agentic system.
- The goal will drive everything else that happens in the workflow.

Clarify the user's goal:
- You'll want to ask two clarifying questions:
- What kind of graph do you want to build?
- What is the purpose of the graph?


## Agent Details

- A goal-oriented, conversational agent that helps the user ideate on the kind of graph to build.
- Input: nothing
- Output: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Tools: `set_perceived_user_goal`, `approve_perceived_user_goal`


## Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [ ]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_5 = "openai/gpt-5"

llm = LiteLlm(model=MODEL_GPT_5)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI is ready!")

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

---

## Define the User Intent Agent (`user_intent_agent`)

### agent instructions



### script

- user intentions set the overall direction for knowledge graph construction
- they are so important that we will preserve them in memory so each step of the workflow is in agreement

In [ ]:
user_goal_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."
"""

In [ ]:
graph_use_case_suggestions = """
    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music
"""

In [ ]:
user_intent_agent_role_and_goal = """
    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.
    Knowledge graph use cases appear in all industries. Wherever there is data, there's probably a graph.
"""

In [ ]:
chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""


In [ ]:
# combine all the instruction components into one complete instruction...
user_intent_agent_instruction = f"""
{user_intent_agent_role_and_goal}
{graph_use_case_suggestions}
{user_goal_definition}
{chain_of_thought_directions}
"""

print(user_intent_agent_instruction)


In [ ]:
#  Define the tools for the User Intent Agent

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)


In [ ]:
#  Define the tools for the User Intent Agent

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(tool_context: ToolContext):
    """Upon approval from user, will record the perceived user goal as the approved user goal."""
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Ask the user to clarify their goal (kind of graph and description).")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state.get(PERCEIVED_USER_GOAL)

    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])


In [ ]:

# add the tools to a list
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

In [ ]:
# Finally, construct the agent

user_intent_agent = Agent(
    name="user_intent_agent_v1",
    model=llm, # defined earlier in a variable
    description="Helps the user ideate on a knowledge graph use case.",
    instruction=user_intent_agent_instruction,
    tools=user_intent_agent_tools,
)

print(f"Agent '{user_intent_agent.name}' created.")

---

## Interact with the Agent



In [ ]:
# Define an Agent Caller Utility
# This will provide a simple "call" interface and access to the session

from helper import make_agent_caller

user_intent_caller = await make_agent_caller(user_intent_agent)

In [ ]:
# Run the Initial Conversation

session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}")

# We need an async function to await our interaction helper
async def run_conversation():
    await user_intent_caller.call("I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, which can illustrate supply chain analysis.", True)
    # Presume approval
    await user_intent_caller.call("Approve that goal.", True)

await run_conversation()

session_end = await user_intent_caller.get_session()

print(f"\nSession End...")


session_end = await user_intent_caller.get_session()

print(f"Session Start: {session_end.state}")

print(f"\nPerceived goal:", session_end.state[PERCEIVED_USER_GOAL])

print(f"\nApproved goal:", session_end.state[APPROVED_USER_GOAL])


---

Congratulations\! You've created a basic human-in-the-loop interaction, with a structured result.

Take a close look at the session state.

- session state starts empty, as you'd expect since we did not initialize it
- after the conversation, there are two values in session state: `perceived_user_goal` and `approved_user_goal`
- this duplication is intentional, separating "working memory" from work specification
- subsequent steps in the workflow should only use work specifications
- in production, work specifications should be persisted to enable tracing and reproducibility


---
## Bonus, An Interactive Conversation

Now, let's make this interactive so you can ask your own questions! Run the cell below. It will prompt you to enter your queries directly.

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await user_intent_caller.call(user_query, True)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()